In [1]:
from pylab import *
import os
import pandas as pd
import librosa
import librosa.display
from joblib import load

In [2]:

def input_parser(input_file):
   try:
      X, sample_rate = librosa.load(input_file, res_type='kaiser_fast') 
      mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=128).T,axis=0) 
   except Exception as e:
      print("Error encountered while parsing file: ", input_file)
      return None
   feature = mfccs.tolist()
   
   return feature

In [3]:
def model_test(input_file):
    results_dict = {
    "predictedEmotion": [],
    "emotionCategories": [], 
    "probabilities": [], 
    "predictedSex": []
    }
    
    model = load('../models/rf2_model.sav')
    model2 = load('../models/gen_emo_rf_model.sav')
    feature = input_parser(input_file)
    arr = np.array(feature)
    arr2d = np.reshape(arr, (1,128))
    pred_emotion = model.predict(arr2d)  
    probs = model.predict_proba(arr2d)
    emotion_labels = model.classes_
    gender = model2.predict(arr2d)
    if gender[0] == 0:
        label = "Male"
    elif gender[0] == 1:
        label = "Female"
    results_dict["predictedEmotion"] = pred_emotion[0]
    results_dict["emotionCategories"] = emotion_labels.tolist()
    results_dict["probabilities"] = probs[0].tolist()
    results_dict["predictedSex"] = label
    return results_dict

In [4]:
audio_file_paths = [
    "../static/audio/mono_a13_hanks.wav", 
    "../static/audio/mono_clue_mustard.wav",
    "../static/audio/mono_ewdavid.wav",
    "../static/audio/mono_meangirls_gretchen.wav",
    "../static/audio/mono_nathan.wav",
    "../static/audio/mono_starwars_vader.wav",
    "../static/audio/mono_theoffice_michael.wav",
    "../static/audio/mono_wizardofoz_witch.wav"
]
tv_movie_sounds = []
keywords = ["hanks", "mustard", "ewdavid", "gretchen", "nathan", "vader", "theoffice", "witch"]
for i in range(len(audio_file_paths)):
    a = model_test(audio_file_paths[i])
    a["keyword"] = keywords[i]
    tv_movie_sounds.append(a.copy())

print(tv_movie_sounds)

[{'predictedEmotion': 'happy', 'emotionCategories': ['angry', 'disgust', 'fearful', 'happy', 'sad', 'surprise'], 'probabilities': [0.15, 0.145, 0.215, 0.28, 0.075, 0.135], 'predictedSex': 'Female', 'keyword': 'hanks'}, {'predictedEmotion': 'angry', 'emotionCategories': ['angry', 'disgust', 'fearful', 'happy', 'sad', 'surprise'], 'probabilities': [0.265, 0.07, 0.09, 0.245, 0.15, 0.18], 'predictedSex': 'Male', 'keyword': 'mustard'}, {'predictedEmotion': 'surprise', 'emotionCategories': ['angry', 'disgust', 'fearful', 'happy', 'sad', 'surprise'], 'probabilities': [0.205, 0.07, 0.105, 0.19, 0.125, 0.305], 'predictedSex': 'Female', 'keyword': 'ewdavid'}, {'predictedEmotion': 'fearful', 'emotionCategories': ['angry', 'disgust', 'fearful', 'happy', 'sad', 'surprise'], 'probabilities': [0.185, 0.08, 0.39, 0.18, 0.075, 0.09], 'predictedSex': 'Male', 'keyword': 'gretchen'}, {'predictedEmotion': 'sad', 'emotionCategories': ['angry', 'disgust', 'fearful', 'happy', 'sad', 'surprise'], 'probabilitie

In [5]:
import json
with open('../static/js/actor_emotions.json', 'w') as file:
    json.dump(tv_movie_sounds, file)